In [2]:
import numpy as np
from tqdm import tqdm
import matplotlib.pylab as plt
import matplotlib.animation as animation
%matplotlib notebook

In [3]:
J = -1.0

In [4]:
def spin_interaction(sconfig, i, j):
    idx = i+1 if i+1 < sconfig.shape[0] else 0
    jdx = j+1 if j+1 < sconfig.shape[1] else 0
    return J*np.dot(sconfig[i][j], (sconfig[idx][j] + sconfig[i][jdx] + sconfig[i-1][j] + sconfig[i][j-1]))

def energy(sconfig):
    nrj = 0
    for j in range(sconfig.shape[0]):
        for i in range(sconfig.shape[1]):
            idx = i+1 if i+1 < sconfig.shape[0] else 0
            jdx = j+1 if j+1 < sconfig.shape[1] else 0
            nrj += J*np.dot(sconfig[i][j], (sconfig[idx][j] + sconfig[i][jdx]))
    return nrj

def magnetization(sconfig):
    return np.sum(sconfig) / float(sconfig.shape[0]*sconfig.shape[1])

def metropolis(sconfig, temp):
    mat = sconfig.copy()
    for i in range(sconfig.shape[0]):
        for j in range(sconfig.shape[1]):
            randvec = np.random.rand(1,3)[0]*((-1)**np.random.randint(0, 2))/10.0
            e0 = spin_interaction(mat, i, j)
            old = mat[i][j].copy()
            mat[i][j] += randvec
            mat[j][i] /= np.linalg.norm(mat[j][i])
            e1 = spin_interaction(mat, i, j)
            if (e1-e0) < 0:
                pass
            else:
                if np.random.rand() <= np.exp(-(e1-e0)/temp):
                    mat[j][i] = old
    return mat


def run(sconfig, temp, max_epoch, verbose=False):
    mags = list()
    energies = list()
    mean_energy = 0
    mean_mag = 0
    lattices = list()
    new = metropolis(sconfig, temp)
    for t in tqdm(xrange(max_epoch), disable = not verbose):
        a = metropolis(new, temp)
        mags.append(magnetization(a))
        energies.append(energy(a))
        new = a
        lattices.append(new)
    return lattices, energies, mags

In [5]:
init_spins = np.random.rand(8,8,3)
#init_spins = np.ones((64, 64, 3))

temperatures = np.arange(0.5, 5, 0.2)

In [6]:
lattices, E, M = run(init_spins, temp=2.0, max_epoch=10000, verbose=True)

100%|██████████| 10000/10000 [00:32<00:00, 309.92it/s]


In [ ]:
fig = plt.figure(figsize=(6,6))
plt.title('Iteration = 0')
im = plt.imshow(lattices[0], interpolation='bilinear')

def update(j):
    im.set_array(lattices[j])
    plt.title('Iteration = {}'.format(j))
    return im

ani = animation.FuncAnimation(fig, update, frames=len(lattices), blit=True)

In [ ]:
plt.figure(figsize=(9,5))
plt.subplot(121)
plt.plot(np.arange(len(M)), M, 'g--')
plt.title('Convergence of magnetization')
plt.xlabel('Time')
plt.ylabel('Mag')

plt.subplot(122)
plt.plot(np.arange(len(E)), E, 'r--')
plt.title('Convergence of energy')
plt.xlabel('Time')
plt.ylabel('Energy')

plt.show()

In [ ]:
configs = list()
E_ls = list()
M_ls = list()
for t in tqdm(temperatures):
    new_spins, E, M = run(init_spins, temp=t, max_epoch=10000)
    configs.append(new_spins)
    E_ls.append(E)
    M_ls.append(M)

In [ ]:
plt.figure(figsize=(9,5))
plt.subplot(121)
plt.plot(temperatures, [np.mean(E_ls[i]) for i in range(len(E_ls))], 'rs--')
plt.title('Mean Energy')
plt.xlabel('Temperature')
plt.ylabel('Energy')


plt.subplot(122)
plt.plot(temperatures, [np.mean(M_ls[i]) for i in range(len(M_ls))], 'gs--')
plt.title('Mean Mags')
plt.xlabel('Temperature')
plt.ylabel('Mag')
plt.show()